In [1]:
from doubt import Boot
from doubt.datasets import (Airfoil, Blog, Concrete, CPU, 
                            FacebookComments, FishBioconcentration,
                            FishToxicity, ForestFire, NewTaipeiHousing,
                            PowerPlant, Protein, Servo,
                            SpaceShuttle)
from mapie.regression import MapieRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import (LinearRegression, PoissonRegressor, 
                                  GammaRegressor, HuberRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from collections import defaultdict

## Setting up

In [2]:
def evaluate_nasa(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    # Calculate training residuals
    model.fit(X_tr, y_tr)
    tr_preds = model.predict(X_tr)
    te_preds = model.predict(X_te)
    tr_residuals = y_tr - tr_preds
    
    n_train = X_tr.shape[0]
    n_test = X_te.shape[0]

    # Initialise random number generator
    rng = np.random.default_rng(4242)

    # Compute the model variances
    bootstrap_preds = np.empty((n_boots, n_test))
    for boot_idx in range(n_boots):
        train_idxs = rng.choice(range(n_train), size=n_train, replace=True)
        X_btr = X_tr[train_idxs, :]
        y_btr = y_tr[train_idxs]

        model.fit(X_btr, y_btr)
        
        bootstrap_pred = model.predict(X_te)
        bootstrap_preds[boot_idx] = bootstrap_pred

    # Centre the bootstrapped predictions across the bootstrap dimension
    bootstrap_preds = np.mean(bootstrap_preds, axis=0) - bootstrap_preds

    # Add up the bootstrap predictions and the hybrid train/val residuals
    C = np.array([m + o for m in bootstrap_preds for o in tr_residuals])

    # Calculate the intervals
    intervals = np.expand_dims(te_preds, -1) + np.transpose(np.quantile(C, q=[uncertainty/2, 1-uncertainty/2], axis=0))
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [3]:
def evaluate_doubt(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    bmodel = Boot(model, random_seed=4242)
    bmodel.fit(X_tr, y_tr, n_boots=n_boots)
    preds, intervals = bmodel.predict(X_te, uncertainty=uncertainty, n_boots=n_boots)
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [4]:
def evaluate_mapie(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    bmodel = MapieRegressor(model)
    bmodel.fit(X_tr, y_tr)
    preds, intervals = bmodel.predict(X_te, alpha=uncertainty)
    
    coverage = np.mean((y_te > intervals[:, 0, 0]) & (y_te < intervals[:, 1, 0]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [5]:
datasets = []
scaler = StandardScaler()

# Add Doubt datasets
dataset_classes = [
    Airfoil,
    Concrete,
    FishToxicity,
    ForestFire,
    NewTaipeiHousing,
    PowerPlant,
    Protein,
    Servo,
]

for dataset_class in dataset_classes:
    dataset = dataset_class()
    dataset._data = dataset._data.sample(n=min(len(dataset), 10000), random_state=4242)
    X_tr, X_te, y_tr, y_te = dataset.split(test_size=0.1, random_seed=4242)
    X_tr = scaler.fit_transform(X_tr)
    X_te = scaler.transform(X_te)
    datasets.append((dataset_class.__name__, X_tr, X_te, y_tr, y_te))
    
len(datasets)

8

In [6]:
for name, X_tr, X_te, y_tr, y_te in datasets:
    print(f'{name}: {len(X_tr) + len(X_te):,} samples, {X_tr.shape[-1]:,} features')

Airfoil: 1,503 samples, 5 features
Concrete: 1,030 samples, 8 features
FishToxicity: 908 samples, 6 features
ForestFire: 517 samples, 12 features
NewTaipeiHousing: 414 samples, 6 features
PowerPlant: 9,568 samples, 4 features
Protein: 10,000 samples, 9 features
Servo: 167 samples, 4 features


## Linear Regression

In [7]:
model = LinearRegression()

In [8]:
nasa_coverages = list()
nasa_mean_widths = list()
doubt_coverages = list()
doubt_mean_widths = list()

data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm([0.01, 0.05, 0.1], leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
linreg_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
linreg_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset          uncertainty                                              
Airfoil          0.01                   -0.280576             -0.280576   
                 0.05                    0.755396              0.035971   
                 0.10                    0.071942              0.791367   
Concrete         0.01                   -1.000000             -1.000000   
                 0.05                   -1.261682             -1.261682   
                 0.10                    2.149533              3.084112   
FishToxicity     0.01                    0.075269              0.075269   
                 0.05                   -1.774194             -2.849462   
                 0.10                   -1.397849             -3.548387   
ForestFire       0.01                   -1.000000              0.923077   
                 0.05                   -3.076923             -3.076923   
                 0.10                   -2.307692             -2.307692   
NewTaipeiHousing 0.01                   -1.000000             -1.000000   
                 0.05                   -2.560976             -0.121951   
                 0.10                   -2.682927             -2.682927   
PowerPlant       0.01                   -0.032258              0.182796   
                 0.05                    0.483871              0.376344   
                 0.10                    1.397849              0.860215   
Protein          0.01                    0.132853             -0.176107   
                 0.05                    0.046344             -0.056643   
                 0.10                    1.431514              1.328527   
Servo            0.01                   -1.000000              6.692308   
                 0.05                    2.692308              2.692308   
                 0.10                   20.769231             13.076923   

                              mapie_coverage_error  nasa_mean_width  \
dataset          uncertainty                                          
Airfoil          0.01                     0.438849        28.146645   
                 0.05                     0.035971        19.354946   
                 0.10                     0.791367        15.483973   
Concrete         0.01                    -1.000000        53.825790   
                 0.05                    -0.327103        42.502705   
                 0.10                     0.280374        35.123406   
FishToxicity     0.01                    -1.000000         5.890626   
                 0.05                    -2.849462         4.040719   
                 0.10                    -3.548387         3.057105   
ForestFire       0.01                     0.923077       308.484358   
                 0.05                     0.769231       106.478481   
                 0.10                     5.384615        70.272444   
NewTaipeiHousing 0.01                    -1.000000        51.299282   
                 0.05                    -0.121951        34.991851   
                 0.10                    -2.682927        23.800120   
PowerPlant       0.01                    -0.139785        23.130679   
                 0.05                     0.806452        16.542828   
                 0.10                     0.860215        14.014139   
Protein          0.01                     0.029866        24.163539   
                 0.05                     0.767250        18.833721   
                 0.10                     1.019567        16.128179   
Servo            0.01                     6.692308         6.325876   
                 0.05                    18.076923         3.529766   
                 0.10                    13.076923         3.177570   

                              doubt_mean_width  mapie_mean_width  
dataset          uncertainty                                      
Airfoil          0.01                30.217236         27.872110  
                 0.05          

In [9]:
error_cols = [col for col in linreg_df.columns if 'error' in col]
(linreg_df[error_cols].abs()
                      .describe()
                      .loc[['mean', 'std']]
                      .T
                      .sort_values(by='mean'))

,mean,std
doubt_coverage_error,2.020065,2.837663
nasa_coverage_error,2.057549,4.095529
mapie_coverage_error,2.609275,4.396513


## Decision Tree

In [10]:
model = DecisionTreeRegressor()

In [11]:
nasa_coverages = list()
nasa_mean_widths = list()
doubt_coverages = list()
doubt_mean_widths = list()

data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm([0.01, 0.05, 0.1], leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
tree_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
tree_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset          uncertainty                                              
Airfoil          0.01                   10.510791             -1.000000   
                 0.05                    5.071942             -1.402878   
                 0.10                    5.107914             -4.964029   
Concrete         0.01                   11.149533             -1.000000   
                 0.05                    9.018692             -3.130841   
                 0.10                   13.364486             -6.261682   
FishToxicity     0.01                   18.354839              2.225806   
                 0.05                   15.430108              3.602151   
                 0.10                   24.408602              6.129032   
ForestFire       0.01                   31.692308             -1.000000   
                 0.05                   37.307692              2.692308   
                 0.10                   34.230769              5.384615   
NewTaipeiHousing 0.01                   16.073171              1.439024   
                 0.05                   43.780488              2.317073   
                 0.10                   31.463415              4.634146   
PowerPlant       0.01                   11.365591             -0.247312   
                 0.05                   13.387097             -0.268817   
                 0.10                   14.408602             -0.860215   
Protein          0.01                   15.683831              0.235839   
                 0.05                   15.082389              1.797116   
                 0.10                   14.098867              2.152420   
Servo            0.01                   22.076923             -1.000000   
                 0.05                   25.769231             10.384615   
                 0.10                   13.076923             13.076923   

                              mapie_coverage_error  nasa_mean_width  \
dataset          uncertainty                                          
Airfoil          0.01                    -1.000000         8.827230   
                 0.05                    -4.280576         8.830468   
                 0.10                    -8.561151         6.880950   
Concrete         0.01                    -1.000000        22.579963   
                 0.05                    -5.000000        22.106012   
                 0.10                    -7.196262        15.572121   
FishToxicity     0.01                    -1.000000         3.244373   
                 0.05                    -1.774194         2.903684   
                 0.10                    -4.623656         2.273784   
ForestFire       0.01                    -1.000000       130.076072   
                 0.05                    -3.076923       114.719615   
                 0.10                    -2.307692        63.477128   
NewTaipeiHousing 0.01                    -1.000000        17.931693   
                 0.05                    -2.560976        15.859756   
                 0.10                    -5.121951        15.328659   
PowerPlant       0.01                    -0.569892        15.514624   
                 0.05                    -4.032258        11.872161   
                 0.10                    -7.741935         9.949000   
Protein          0.01                    -0.794027        16.045532   
                 0.05                    -4.279094        13.293073   
                 0.10                    -7.425335        11.756957   
Servo            0.01                    -1.000000         1.460597   
                 0.05                     2.692308         1.399055   
                 0.10                    13.076923         1.460597   

                              doubt_mean_width  mapie_mean_width  
dataset          uncertainty                                      
Airfoil          0.01                18.712431         19.223662  
                 0.05          

In [12]:
error_cols = [col for col in linreg_df.columns if 'error' in col]
(tree_df[error_cols].abs()
                    .describe()
                    .loc[['mean', 'std']]
                    .T
                    .sort_values(by='mean'))

,mean,std
doubt_coverage_error,3.216952,3.227193
mapie_coverage_error,3.796465,3.158507
nasa_coverage_error,18.829758,10.324322
